# 工商時報的共矩陣

In [1]:
import os
import jieba
import numpy as np
from collections import Counter
# pip install gensim
from gensim.corpora import Dictionary

paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress


### 讀入stopwords

In [2]:
stopwords = []
jieba.set_dictionary('./jieba_data/dict.txt.big')
jieba.load_userdict('./jieba_data/userdict.txt')

with open('./jieba_data/stopwords.txt', 'r', encoding='UTF-8') as file:
    for data in file.readlines():
        data = data.strip()
        stopwords.append(data)

Building prefix dict from C:\Users\鄒峻安\Desktop\github project\HW1--data-mining\HW2\jieba_data\dict.txt.big ...
Loading model from cache C:\Users\鄒峻安\AppData\Local\Temp\jieba.uea1b80d53a35cd1eaa4d428049c7a876.cache
Loading model cost 1.284 seconds.
Prefix dict has been built succesfully.


In [3]:
def getTokens(text):
    tokens = []
    if len(text) > 0:
        segments = jieba.cut(text,cut_all=False)
        for t in segments:
            t = t.strip()
            if not t in stopwords:
                tokens.append(t)
    return tokens

In [4]:
def getText(filename):
    with open(filename,'r', encoding='UTF-8') as file:
        lines = file.readlines()
        
    news_title = lines[2]
    news_body = lines[3]
    text = news_title + news_body
    return(text)    

### 取出全部文件(180個)的內容做斷詞

In [5]:
text = ''
dir_path = './ctee_TMC'
for file in os.listdir(dir_path):
    file_path = os.path.join(dir_path,file)
    if os.path.isfile(file_path) and not file.startswith('.'):
        text = text + getText(file_path)

tokens = getTokens(text)

### 共有約2萬個詞

In [6]:
print(len(tokens))

19702


### 共有約5000個不重複詞

In [7]:
counter = Counter(tokens)
print(len(counter))

5162


### 取出詞頻最高前500個詞

In [8]:
#top_tokens_tuple = (字詞,出現次數)
top_tokens_tuple = counter.most_common(500)
all_tokens = []
for t in top_tokens_tuple:
    all_tokens.append(t[0])

print(all_tokens)

['|', '台積電', '元', '工商時報', '億元', '台積', '％', '奈米', '營收', '日', '外資', '月', '製程', '今日焦點', '積電', '市場', ',', '晶圓', '台股', '法人', '成長', '去年', '科技', '股價', '代工', '10', '年', '脈動', '股利', '(', ')', '預期', '龍頭', '半導體', '現金', '2330', '約', '點', '買超', '新高', '後', '第一季', '台灣', '-', '高', '中', '市值', '晶片', '蘋果', '台', '影響', '指出', '產業', '達', '技術', '會', '大', '先進', '新台幣', '可望', '客戶', '～', '上漲', '公司', '仍', '每股', '持續', '設計', '營運', '昨', '第四季', '全球', '說', '前', '投資', '獲利', '證券', '產能', '董事長', '都', '手機', '歷史', '預估', '訂單', '指數', '合併', '12', '處理器', '新', '股', '張忠謀', '未來', '大立光', '董事會', '明年', '廠', '減少', '個股', '已', '億美元', '2019年', '展望', '包括', '帶動', '更', '逾', '財經', '2018年', '毛利率', '上', '不', '季', '發展', '美元', '表現', '增加', '聯發科', '全年', '受惠', '需求', '業績', '出貨', '兆元', '2020年', '產品', '下', 'iPhone', '漲幅', '行情', '即時新聞', '事件', '創新', '盤中', '第二季', '人', '5G', '創意', '配發', '營收達', '大陸', '看好', '再', '量產', '震盪', '季減', '稅後', '紀錄', '英特爾', '優於', '占', '11', '企業', '16', '營業', '淨利', '資金', '11月', '去年同期', '運算', '動能', '權值', '評等', '比特', '加上', '發放', '張', '鴻

### 過濾掉沒有在top 500裡的詞

In [9]:
def filterTokens(tokens):
    global all_tokens
    filtered_tokens = []
    for t in tokens:
        if t in all_tokens:
            filtered_tokens.append(t)
    return filtered_tokens

In [10]:
doc_tokens = []
dir_path = './ctee_TMC'
for file in os.listdir(dir_path):
    file_path = os.path.join(dir_path,file)
    if os.path.isfile(file_path) and not file.startswith('.'):
        wanted_tokens = []
        text = getText(file_path)
        tokens = getTokens(text)
        tokens2 = filterTokens(tokens)
        doc_tokens.append(tokens2)

### 稀疏矩陣

In [11]:
sparse_matrix = []

dir_path = './ctee_TMC'
for file in os.listdir(dir_path):
    file_path = os.path.join(dir_path,file)
    if os.path.isfile(file_path) and not file.startswith('.'):
        text = getText(file_path)
        tokens = getTokens(text)
        doc_to_term = []
        for t in all_tokens:
            if t in tokens:
                doc_to_term.append(1)
            else:
                doc_to_term.append(0)
        sparse_matrix.append(doc_to_term)

### 共現矩陣

In [12]:
def co_occurrence_matrix(ls):
    global matrix
    length = len(ls)
    for i in range(length):
        for j in range(length):
            if i != j:
                #同一篇文章中 單詞和除了本身的單詞 出現次數+1
                matrix[[ls[i]], [ls[j]]] += 1

In [13]:
dimension = len(all_tokens)
matrix = np.matrix([[0] * dimension] * dimension)

dt = Dictionary(doc_tokens).token2id

# dt[doc_tokens[0][1]] = dt['領先']
# 將文字經過dt換成所代表數字
doc_token_ids = [[dt[word] for word in words] for words in doc_tokens]

for ids in doc_token_ids:
    co_occurrence_matrix(ids)

In [14]:
print(dt)

{',': 0, '000': 1, '|': 2, '上': 3, '世代': 4, '今日焦點': 5, '代工': 6, '億元': 7, '先進': 8, '創新': 9, '可望': 10, '台積': 11, '台積電': 12, '奈米': 13, '工商時報': 14, '布局': 15, '技術': 16, '持續': 17, '支援': 18, '產能': 19, '科技': 20, '脈動': 21, '製程': 22, '資金': 23, '電將': 24, '領先': 25, '龍頭': 26, '11月': 27, '個股': 28, '出貨': 29, '創下': 30, '受惠': 31, '合併': 32, '單季': 33, '大廠': 34, '新高': 35, '晶圓': 36, '機會': 37, '歷史': 38, '法人': 39, '營收': 40, '營收達': 41, '產業': 42, '目標': 43, '第四季': 44, '紀錄': 45, '蘋果': 46, '處理器': 47, '達': 48, '量': 49, '順利': 50, '高': 51, '今': 52, '台灣': 53, '張': 54, '張忠謀': 55, '營運': 56, '看': 57, '董事長': 58, '說': 59, '10': 60, '不': 61, '仍': 62, '供應鏈': 63, '億': 64, '公司': 65, '卻': 66, '積電': 67, '員工': 68, '報導': 69, '大陸': 70, '好': 71, '近期': 72, '都': 73, '(': 74, ')': 75, '2330': 76, 'ADR': 77, '元': 78, '前': 79, '台股': 80, '年': 81, '股價': 82, '中': 83, '兌': 84, '匯率': 85, '去年': 86, '大關': 87, '季': 88, '已': 89, '市場': 90, '平均': 91, '影響': 92, '新台幣': 93, '昨': 94, '會': 95, '月': 96, '法': 97, '約': 98, '美元匯率': 99, '表現': 100, '電在': 101

In [15]:
print(matrix)

[[ 314   16  236 ...    0    0    0]
 [  16    4   24 ...    0    0    0]
 [ 236   24 1812 ...   44   32    0]
 ...
 [   0    0   44 ...   62    0    0]
 [   0    0   32 ...    0   56    0]
 [   0    0    0 ...    0    0   72]]


In [63]:
top_tokens_tuple[0]

('|', 604)

In [70]:
#查詢台積電出現次數
dict(top_tokens_tuple)["台積電"]

435

In [76]:
td = { v: k for k, v in dt.items() }
td

{0: ',',
 1: '000',
 2: '|',
 3: '上',
 4: '世代',
 5: '今日焦點',
 6: '代工',
 7: '億元',
 8: '先進',
 9: '創新',
 10: '可望',
 11: '台積',
 12: '台積電',
 13: '奈米',
 14: '工商時報',
 15: '布局',
 16: '技術',
 17: '持續',
 18: '支援',
 19: '產能',
 20: '科技',
 21: '脈動',
 22: '製程',
 23: '資金',
 24: '電將',
 25: '領先',
 26: '龍頭',
 27: '11月',
 28: '個股',
 29: '出貨',
 30: '創下',
 31: '受惠',
 32: '合併',
 33: '單季',
 34: '大廠',
 35: '新高',
 36: '晶圓',
 37: '機會',
 38: '歷史',
 39: '法人',
 40: '營收',
 41: '營收達',
 42: '產業',
 43: '目標',
 44: '第四季',
 45: '紀錄',
 46: '蘋果',
 47: '處理器',
 48: '達',
 49: '量',
 50: '順利',
 51: '高',
 52: '今',
 53: '台灣',
 54: '張',
 55: '張忠謀',
 56: '營運',
 57: '看',
 58: '董事長',
 59: '說',
 60: '10',
 61: '不',
 62: '仍',
 63: '供應鏈',
 64: '億',
 65: '公司',
 66: '卻',
 67: '積電',
 68: '員工',
 69: '報導',
 70: '大陸',
 71: '好',
 72: '近期',
 73: '都',
 74: '(',
 75: ')',
 76: '2330',
 77: 'ADR',
 78: '元',
 79: '前',
 80: '台股',
 81: '年',
 82: '股價',
 83: '中',
 84: '兌',
 85: '匯率',
 86: '去年',
 87: '大關',
 88: '季',
 89: '已',
 90: '市場',
 91: '平均',
 92: '影

In [78]:
import pandas as pd
pd.DataFrame(matrix).rename(td)

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
",",314,16,236,47,1,41,53,358,22,15,...,6,45,3,11,1,4,9,0,0,0
000,16,4,24,1,1,3,6,14,1,1,...,0,0,0,0,0,0,0,0,0,0
|,236,24,1812,32,28,432,256,424,144,76,...,8,4,8,0,28,16,0,44,32,0
上,47,1,32,28,9,5,8,64,12,8,...,2,29,4,0,0,0,22,0,0,9
世代,1,1,28,9,8,2,5,5,14,7,...,0,0,0,0,0,0,0,0,0,0
今日焦點,41,3,432,5,2,0,47,49,19,10,...,0,0,0,0,0,0,0,0,0,0
代工,53,6,256,8,5,47,26,92,42,10,...,5,4,4,0,0,0,1,0,0,0
億元,358,14,424,64,5,49,92,728,27,9,...,24,39,18,0,4,12,60,0,8,0
先進,22,1,144,12,14,19,42,27,44,11,...,0,0,3,0,2,4,0,0,0,0
創新,15,1,76,8,7,10,10,9,11,14,...,0,3,0,11,1,4,0,0,0,0
